# Combining IR and RGB models

In [1]:
%cd ~/code/saoriyosano/poverty_mapper

/home/saori/code/saoriyosano/poverty_mapper


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from glob import glob
from batchers import dataset_constants, tfrecord_paths_utils
from models import processing
from models.loss import r2
from models.checkpoint import CustomModelCheckpoint
import datetime

2023-03-10 13:46:20.199962: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 13:46:23.126947: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-10 13:46:23.127020: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-10 13:46:23.266567: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


KeyboardInterrupt: 

---
### Target

In [30]:
df = pd.read_csv('data/dhs_clusters.csv')

In [31]:
y = df[(df.country=='angola')&(df.year==2011)].head(40).wealthpooled.values

In [32]:
y

array([ 2.59561753,  2.20962048,  0.90646887,  1.1053592 ,  1.87934387,
        1.74931741,  0.18236142,  0.69121599,  0.527798  ,  0.89116049,
        0.936777  , -0.05265219, -0.07346991,  0.72167271, -0.11726858,
        0.38161853, -1.14221025, -0.85293049, -0.46350324, -1.0571233 ,
       -1.0633744 ,  0.4095335 , -0.90661067, -1.17829394, -1.04396415,
       -1.13135445,  0.15251048, -0.81384754, -0.43085682, -0.24942476,
       -0.83738708, -0.93222094, -0.94360435, -1.14929593, -1.0485065 ,
       -0.82958251, -0.62332767, -0.94889927, -0.95115322, -1.06607962])

---  
### Creating dataset - no shuffling

In [ ]:
# DHS_TFRECORDS_PATH_ROOT = 'data/dhs_tfrecords/' #local
DHS_TFRECORDS_PATH_ROOT = 'gcs/dhs_tfrecords/' #VM
CSV_PATH = 'data/dhs_clusters.csv' 
CHECKPOINT_PATH = 'models/checkpoints/' 

CLUSTERS_DF = pd.read_csv(CSV_PATH, float_precision='high', index_col=False)
MEANS = dataset_constants._MEANS_DHS
STDS = dataset_constants._STD_DEVS_DHS
BATCH_SIZE = 8
DATASET = 'DHS_OOC_A'
SHUFFLE = 16
PREFETCH = 2
EPOCHS = 100 # CHANGE
STEPS_PER_EPOCH = 1474
VALIDATION_STEPS = 488

In [ ]:
def process_datasets_ir(dataset, shuffle, batch_size, prefetch, epochs):
    ''' prepares train_ds, val_ds and test_ds'''
    
    train_tfrecord_paths = tfrecord_paths_utils.dhs_ooc(dataset, split="train")
    val_tfrecord_paths = tfrecord_paths_utils.dhs_ooc(dataset, split="val")
    test_tfrecord_paths = tfrecord_paths_utils.dhs_ooc(dataset, split="test")
    
    # for testing - comment out
#     train_tfrecord_paths = train_tfrecord_paths[:300]
#     val_tfrecord_paths = val_tfrecord_paths[300:400]
#     test_tfrecord_paths = test_tfrecord_paths[0:1]
    print(len(train_tfrecord_paths), len(val_tfrecord_paths), len(test_tfrecord_paths))

    train_ds = tf.data.TFRecordDataset(train_tfrecord_paths, compression_type="GZIP")
    val_ds = tf.data.TFRecordDataset(val_tfrecord_paths, compression_type="GZIP")
    test_ds = tf.data.TFRecordDataset(test_tfrecord_paths, compression_type="GZIP")

    # normalize and resize
    train_ds = train_ds.map(processing.process_tfrecords)
    val_ds = val_ds.map(processing.process_tfrecords)  
    test_ds = test_ds.map(processing.process_tfrecords)

    # train_ds = train_ds.map(processing.augment)
    # val_ds = val_ds.map(processing.augment)  
    # test_ds = test_ds.map(processing.augment)

    train_ds = train_ds.cache()
#     train_ds = train_ds.shuffle(shuffle)
    train_ds = train_ds.batch(batch_size)
    train_ds = train_ds.repeat(epochs) # repeats the dataset for the number of epochs 
    train_ds = train_ds.prefetch(prefetch)

    val_ds = val_ds.cache()
#     val_ds = val_ds.shuffle(shuffle)
    val_ds = val_ds.batch(batch_size)
    val_ds = val_ds.prefetch(prefetch)

    test_ds = test_ds.cache()
#     test_ds = test_ds.shuffle(shuffle)
    test_ds = test_ds.batch(batch_size)
    test_ds = test_ds.prefetch(prefetch)

    return train_ds, val_ds, test_ds

In [ ]:
ir_train_ds, ir_val_ds, ir_test_ds = process_datasets_ir(DATASET, SHUFFLE, BATCH_SIZE, PREFETCH, EPOCHS)

In [127]:
def process_datasets_rgb(dataset, shuffle, batch_size, prefetch, epochs):
    ''' prepares train_ds, val_ds and test_ds'''
    
    train_tfrecord_paths = tfrecord_paths_utils.dhs_ooc(dataset, split="train")
    val_tfrecord_paths = tfrecord_paths_utils.dhs_ooc(dataset, split="val")
    test_tfrecord_paths = tfrecord_paths_utils.dhs_ooc(dataset, split="test")
    
    # for testing - comment out
#     train_tfrecord_paths = train_tfrecord_paths[:300]
#     val_tfrecord_paths = val_tfrecord_paths[300:400]
#     test_tfrecord_paths = test_tfrecord_paths[0:1]
    print(len(train_tfrecord_paths), len(val_tfrecord_paths), len(test_tfrecord_paths))

    train_ds = tf.data.TFRecordDataset(train_tfrecord_paths, compression_type="GZIP")
    val_ds = tf.data.TFRecordDataset(val_tfrecord_paths, compression_type="GZIP")
    test_ds = tf.data.TFRecordDataset(test_tfrecord_paths, compression_type="GZIP")

    # normalize and resize
    train_ds = train_ds.map(processing.process_tfrecords_rgb)
    val_ds = val_ds.map(processing.process_tfrecords_rgb)  
    test_ds = test_ds.map(processing.process_tfrecords_rgb)

    # train_ds = train_ds.map(processing.augment)
    # val_ds = val_ds.map(processing.augment)  
    # test_ds = test_ds.map(processing.augment)

    train_ds = train_ds.cache()
#     train_ds = train_ds.shuffle(shuffle)
    train_ds = train_ds.batch(batch_size)
    train_ds = train_ds.repeat(epochs) # repeats the dataset for the number of epochs 
    train_ds = train_ds.prefetch(prefetch)

    val_ds = val_ds.cache()
#     val_ds = val_ds.shuffle(shuffle)
    val_ds = val_ds.batch(batch_size)
    val_ds = val_ds.prefetch(prefetch)

    test_ds = test_ds.cache()
#     test_ds = test_ds.shuffle(shuffle)
    test_ds = test_ds.batch(batch_size)
    test_ds = test_ds.prefetch(prefetch)

    return train_ds, val_ds, test_ds

In [128]:
rgb_train_ds, rgb_val_ds, rgb_test_ds = process_datasets_rgb(DATASET, SHUFFLE, BATCH_SIZE, PREFETCH, EPOCHS)

11797 3909 3963
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


---
# Evaluate on individual models

In [130]:
rgb_path = 'models/checkpoints/MN_RGB_20230307-073729/model_epoch02.h5'
ir_path = 'models/checkpoints/MN_IR_20230307-073301/model_epoch04.h5' # with dropout
RGB_MODEL = tf.keras.models.load_model(rgb_path, custom_objects={'r2': r2})
IR_MODEL = tf.keras.models.load_model(ir_path, custom_objects={'r2': r2})

In [131]:
RGB_MODEL.evaluate(rgb_test_ds.take(10))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
     10/Unknown - 28s 3s/step - loss: 0.7903 - mse: 0.7903 - mae: 0.7452 - r2: -8.2130

2023-03-07 12:48:35.035744: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


10/10 [==============================] - 31s 3s/step - loss: 0.7903 - mse: 0.7903 - mae: 0.7452 - r2: -8.2130


[0.7902749180793762,
 0.7902749180793762,
 0.7451528310775757,
 -8.212976455688477]

In [132]:
IR_MODEL.evaluate(ir_test_ds.take(10))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
     10/Unknown - 25s 2s/step - loss: 0.6034 - mse: 0.6034 - mae: 0.6499 - r2: -4.2713

2023-03-07 12:49:02.929670: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


10/10 [==============================] - 29s 3s/step - loss: 0.6034 - mse: 0.6034 - mae: 0.6499 - r2: -4.2713


[0.603441059589386, 0.603441059589386, 0.6499134302139282, -4.271349906921387]

---
# Combine IR and RGB models - average

In [89]:
def testset_y(dataset):
    survey_names = dataset_constants.SURVEY_NAMES[dataset]
    df = pd.read_csv('data/dhs_clusters.csv')
    y = np.array([])
    for country in survey_names['test']:
        y = np.concatenate(
            (y, df[df.country==country].wealthpooled.values)
        )
    return y

In [94]:
y = testset_y('DHS_OOC_A')

In [95]:
y.shape

(3963,)

In [100]:
def ensemble_mean(rgb_model_path, ir_model_path, rgb_test_ds, ir_test_ds, y):
    # load saved models
    rgb_path = rgb_model_path
    ir_path = ir_model_path
    RGB_MODEL = tf.keras.models.load_model(rgb_path, custom_objects={'r2': r2})
    IR_MODEL = tf.keras.models.load_model(ir_path, custom_objects={'r2': r2})
    
    # predict
    rgb_pred = RGB_MODEL.predict(rgb_test_ds, verbose=1)
    ir_pred = IR_MODEL.predict(ir_test_ds, verbose=1)
    
    # calculate errors
    combined_pred = np.concatenate((rgb_pred, ir_pred), axis=1)
    mean_pred = np.mean(combined_pred, axis=1)
    error = y - mean_pred
    mse = np.mean((error ** 2))
    
    dict = {
        'mse': mse, 
        'rgb_pred': rgb_pred, 
        'ir_pred': ir_pred, 
        'combined_pred': combined_pred
    }
    
    return dict

In [101]:
rgb_path = 'models/checkpoints/MN_RGB_20230307-023713/model_epoch02.h5'
ir_path = 'models/checkpoints/MN_IR_20230307-022906/model_epoch01.h5'

In [102]:
result = ensemble_mean(rgb_path, ir_path, rgb_test_ds.take(10), ir_test_ds.take(10), y[:80])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
     10/Unknown - 24s 2s/step

2023-03-07 08:18:30.471374: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


10/10 [==============================] - 28s 3s/step
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
     10/Unknown - 26s 3s/step

2023-03-07 08:19:00.244563: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


10/10 [==============================] - 30s 3s/step


In [107]:
result['mse']

0.6831756753308482

--- 
# Average with weights

In [121]:
def ensemble_weighted_mean(rgb_model_path, ir_model_path, rgb_test_ds, ir_test_ds, y, rgb_weight, ir_weight):
    # load saved models
    rgb_path = rgb_model_path
    ir_path = ir_model_path
    RGB_MODEL = tf.keras.models.load_model(rgb_path, custom_objects={'r2': r2})
    IR_MODEL = tf.keras.models.load_model(ir_path, custom_objects={'r2': r2})
    
    # predict
    rgb_pred = RGB_MODEL.predict(rgb_test_ds, verbose=1) * rgb_weight
    ir_pred = IR_MODEL.predict(ir_test_ds, verbose=1) * ir_weight
    
    # calculate errors
    combined_pred = np.concatenate((rgb_pred, ir_pred), axis=1)
    mean_pred = np.mean(combined_pred, axis=1)
    error = y - mean_pred
    mse = np.mean((error ** 2))
    
    dict = {
        'mse': mse, 
        'rgb_pred': rgb_pred, 
        'ir_pred': ir_pred, 
        'combined_pred': combined_pred
    }
    
    return dict

In [128]:
result = ensemble_weighted_mean(rgb_path, ir_path, rgb_test_ds.take(10), ir_test_ds.take(10), y[:80], .5, 1.5)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
     10/Unknown - 30s 3s/step

2023-03-07 09:10:08.573957: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


10/10 [==============================] - 34s 3s/step
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
     10/Unknown - 35s 3s/step

2023-03-07 09:10:47.734418: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


10/10 [==============================] - 39s 4s/step


In [129]:
result['mse']

0.6646803655022506

- MSE when RGB weight .6, IR weight 1.4: **0.6653829990818261** (10 batches, 80 images)
- MSE when RGB weight .5, IR weight 1.5: **0.6646803655022506**

---
# Stacking

In [31]:
!pwd

/home/saori/code/saoriyosano/poverty_mapper


In [30]:
last_best = 'model/saved_models/best_model_combined.h5'

In [29]:
rgb_best = tf.keras.models.load_model(
    'model/saved_models/best_model_rgb.h5', 
    custom_objects={'r2': r2}
)
ir_best = tf.keras.models.load_model(
    'model/saved_models/best_model_ir.h5', 
    # 'model/checkpoints/EN_20230306-090956/model_epoch13.h5',
    custom_objects={'r2': r2}
)

In [32]:
# make the two base models untrainable
rgb_best.trainable = False
ir_best.trainable = False

In [33]:
rgb_best.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 images (InputLayer)            [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['images[0][0]']                 
                                                                                                  
 Conv (Conv2D)                  (None, 112, 112, 16  432         ['rescaling[0][0]']              
                                )                                                                 
                                                                                              

In [34]:
ir_best.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 images (InputLayer)            [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['images[0][0]']                 
                                                                                                  
 Conv (Conv2D)                  (None, 112, 112, 16  432         ['rescaling_1[0][0]']            
                                )                                                                 
                                                                                            

### Renaming the layers

In [ ]:
# _config = rgb_do.get_config()
# len(_config['layers'])

In [35]:
def add_prefix(model, prefix: str, custom_objects=None):
    '''Adds a prefix to layers and model name while keeping the pre-trained weights
    Arguments:
        model: a tf.keras model
        prefix: a string that would be added to before each layer name
        custom_objects: if your model consists of custom layers you shoud add them pass them as a dictionary. 
            For more information read the following:
            https://keras.io/guides/serialization_and_saving/#custom-objects
    Returns:
        new_model: a tf.keras model having same weights as the input model.
    '''
    
    config = model.get_config()
    old_to_new = {}
    new_to_old = {}
    i=1
    for layer in config['layers']:
        print(f"layer {i}: {layer['name']} (old name)")
        new_name = prefix + layer['name']
        old_to_new[layer['name']], new_to_old[new_name] = new_name, layer['name']
        layer['name'] = new_name
        layer['config']['name'] = new_name
        
        if len(layer['inbound_nodes']) > 0:
#             print(layer['inbound_nodes'], len(layer['inbound_nodes'][0]), type(layer['inbound_nodes'][0]))
            if np.array(layer['inbound_nodes']).ndim == 2:
                old_name_ = layer['inbound_nodes'][0][0]
                layer['inbound_nodes'][0].remove(layer['inbound_nodes'][0][0])
                print(layer['inbound_nodes'])
                layer['inbound_nodes'][0].insert(0, old_to_new[old_name_])
            else:
                for in_node in layer['inbound_nodes'][0]:
                    in_node[0] = old_to_new[in_node[0]]
        print(layer['inbound_nodes'])
#             else:
#                 layer['inbound_nodes'] = old_to_new[layer['inbound_nodes']]
            
        i+=1
    
    for input_layer in config['input_layers']:
        input_layer[0] = old_to_new[input_layer[0]]
    
    for output_layer in config['output_layers']:
        output_layer[0] = old_to_new[output_layer[0]]
    
    config['name'] = prefix + config['name']
    new_model = tf.keras.Model().from_config(config, custom_objects)
    
    for layer in new_model.layers:
        layer.set_weights(model.get_layer(new_to_old[layer.name]).get_weights())
    
    return new_model

In [36]:
rgb_model = add_prefix(rgb_best, 'rgb', custom_objects={'r2': r2})
rgb_model

layer 1: images (old name)
[]
layer 2: rescaling (old name)
[[['rgbimages', 0, 0, {}]]]
layer 3: Conv (old name)
[[['rgbrescaling', 0, 0, {}]]]
layer 4: Conv/BatchNorm (old name)
[[['rgbConv', 0, 0, {}]]]
layer 5: tf.__operators__.add (old name)
[[0, 0, {'y': 3.0, 'name': None}]]
[['rgbConv/BatchNorm', 0, 0, {'y': 3.0, 'name': None}]]
layer 6: re_lu (old name)
[[['rgbtf.__operators__.add', 0, 0, {}]]]
layer 7: tf.math.multiply (old name)
[[0, 0, {'y': 0.16666666666666666, 'name': None}]]
[['rgbre_lu', 0, 0, {'y': 0.16666666666666666, 'name': None}]]
layer 8: multiply (old name)
[[['rgbConv/BatchNorm', 0, 0, {}], ['rgbtf.math.multiply', 0, 0, {}]]]
layer 9: expanded_conv/depthwise/pad (old name)
[[['rgbmultiply', 0, 0, {}]]]
layer 10: expanded_conv/depthwise (old name)
[[['rgbexpanded_conv/depthwise/pad', 0, 0, {}]]]
layer 11: expanded_conv/depthwise/BatchNorm (old name)
[[['rgbexpanded_conv/depthwise', 0, 0, {}]]]
layer 12: re_lu_1 (old name)
[[['rgbexpanded_conv/depthwise/BatchNorm', 

In [37]:
ir_model = add_prefix(ir_best, 'ir', custom_objects={'r2': r2})
ir_model

layer 1: images (old name)
[]
layer 2: rescaling_1 (old name)
[[['irimages', 0, 0, {}]]]
layer 3: Conv (old name)
[[['irrescaling_1', 0, 0, {}]]]
layer 4: Conv/BatchNorm (old name)
[[['irConv', 0, 0, {}]]]
layer 5: tf.__operators__.add_27 (old name)
[[0, 0, {'y': 3.0, 'name': None}]]
[['irConv/BatchNorm', 0, 0, {'y': 3.0, 'name': None}]]
layer 6: re_lu_32 (old name)
[[['irtf.__operators__.add_27', 0, 0, {}]]]
layer 7: tf.math.multiply_27 (old name)
[[0, 0, {'y': 0.16666666666666666, 'name': None}]]
[['irre_lu_32', 0, 0, {'y': 0.16666666666666666, 'name': None}]]
layer 8: multiply_18 (old name)
[[['irConv/BatchNorm', 0, 0, {}], ['irtf.math.multiply_27', 0, 0, {}]]]
layer 9: expanded_conv/depthwise/pad (old name)
[[['irmultiply_18', 0, 0, {}]]]
layer 10: expanded_conv/depthwise (old name)
[[['irexpanded_conv/depthwise/pad', 0, 0, {}]]]
layer 11: expanded_conv/depthwise/BatchNorm (old name)
[[['irexpanded_conv/depthwise', 0, 0, {}]]]
layer 12: re_lu_33 (old name)
[[['irexpanded_conv/depth

### Building the combined  model

In [38]:
rgb_input = rgb_model.input
rgb_output = rgb_model.output
ir_input = ir_model.input
ir_output = ir_model.output

In [43]:
inputs = [rgb_input, ir_input]
combined = tf.keras.layers.concatenate([rgb_output, ir_output])
x = tf.keras.layers.Dense(
    10, activation='relu',
    kernel_regularizer=tf.keras.regularizers.L2(0.1)
)(combined)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)
model_combined = tf.keras.models.Model(inputs=inputs, outputs=outputs)

In [44]:
model_combined.load_weights(last_best)

In [45]:
model_combined

## Creating data to feed to the model

In [ ]:
# rgb_train = []
# rgb_y_train = []
# i=0
# for el in rgb_train_ds.as_numpy_iterator():
#     rgb_train.append(el[0])
#     rgb_y_train.append(el[1])
#     print(i)
#     i+=1
# rgb_train_2 = np.array(rgb_train).reshape(816, 224, 224, 3)
# # rgb_train_816_tensor = tf.convert_to_tensor(rgb_train_2) 
# np.save('train_rgb_all', rgb_train_2)
# y = np.array(rgb_y_train).reshape(816)
# # y_tensor = tf.convert_to_tensor(y)
# np.save('train_y_all', y)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
# ir_train = []
# ir_y_train = []
# i=0
# for el in ir_train_ds.as_numpy_iterator():
#     ir_train.append(el[0])
#     ir_y_train.append(el[1])
#     print(i)
#     i+=1
# ir_train_2 = np.array(ir_train).reshape(816, 224, 224, 3)
# # ir_train_all_tensor = tf.convert_to_tensor(ir_train_2)
# np.save('train_ir_all', ir_train_2)

In [74]:
# np.save('train_rgb_first816', rgb_train_2)
# np.save('train_ir_first816', ir_train_2)
# np.save('train_y_first816', y)

In [18]:
rgb_data = np.load('train_rgb_first816.npy')
ir_data = np.load('train_ir_first816.npy')
y_data = np.load('train_y_first816.npy')

In [ ]:
# train combined model

now = datetime.datetime.now()
date_time = now.strftime("%Y%m%d-%H%M%S")
checkpoint_path = os.path.join('models/checkpoints', f"combined_{date_time}")
    
if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(checkpoint_path, 'model.h5'),
        save_best_only=True,
        verbose=1
    )
    
    # logs the outcome of every epoch in a csv file
csvpath = os.path.join(checkpoint_path, 'model_history_log.csv')
print(f"csvpath: {csvpath}")
with open(csvpath, 'a', encoding='utf-8') as f:
    csv_logger = tf.keras.callbacks.CSVLogger(
            csvpath,
            separator=',',
            append=True
    )
    
es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=100,
    verbose=1,
    restore_best_weights=True,
)

adam = tf.keras.optimizers.Adam(
        learning_rate=0.0001
    )
model_combined.compile(loss='mse', optimizer=adam, metrics=['mse', 'mae', r2])
# model_combined.summary()

model_combined.fit(
    x = [rgb_data, ir_data], y=y_data,
    validation_split=.3,
    epochs=1000,
    batch_size=32, 
    callbacks=[csv_logger, cp_callback, es],
    verbose=1
    )

csvpath: models/checkpoints/combined_20230310-123741/model_history_log.csv
Epoch 1/1000
18/18 [==============================] - ETA: 0s - loss: 0.5011 - mse: 0.1574 - mae: 0.2940 - r2: 0.7581
Epoch 1: val_loss improved from inf to 0.50628, saving model to models/checkpoints/combined_20230310-123741/model.h5
18/18 [==============================] - 105s 5s/step - loss: 0.5011 - mse: 0.1574 - mae: 0.2940 - r2: 0.7581 - val_loss: 0.5063 - val_mse: 0.1638 - val_mae: 0.3253 - val_r2: 0.5873
Epoch 2/1000
18/18 [==============================] - ETA: 0s - loss: 0.4989 - mse: 0.1575 - mae: 0.2943 - r2: 0.7607
Epoch 2: val_loss improved from 0.50628 to 0.50552, saving model to models/checkpoints/combined_20230310-123741/model.h5
18/18 [==============================] - 34s 2s/step - loss: 0.4989 - mse: 0.1575 - mae: 0.2943 - r2: 0.7607 - val_loss: 0.5055 - val_mse: 0.1652 - val_mae: 0.3278 - val_r2: 0.5836
Epoch 3/1000
18/18 [==============================] - ETA: 0s - loss: 0.4968 - mse: 0.15

---
# garbage

In [118]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression

In [ ]:
# def ensemble_regression(rgb_model_path, ir_model_path, rgb_test_ds, ir_test_ds, y):
#     # load saved models
#     rgb_path = rgb_model_path
#     ir_path = ir_model_path
#     RGB_MODEL = tf.keras.models.load_model(rgb_path, custom_objects={'r2': r2})
#     IR_MODEL = tf.keras.models.load_model(ir_path, custom_objects={'r2': r2})
    
#     # predict
#     rgb_pred = RGB_MODEL.predict(rgb_test_ds, verbose=1)
#     ir_pred = IR_MODEL.predict(ir_test_ds, verbose=1)
    
#     # fit regression
#     combined_pred = np.concatenate((rgb_pred, ir_pred), axis=1)
#     reg = LinearRegression()
#     reg.fit(combined_pred, y)
#     pred = reg.predict(combined_pred)
    
    
#     error = y - mean_pred
#     mse = np.mean((error ** 2))
    
#     dict = {
#         'mse': mse, 
#         'rgb_pred': rgb_pred, 
#         'ir_pred': ir_pred, 
#         'combined_pred': combined_pred
#     }
    
#     return dict

In [ ]:
result = ensemble_regression(rgb_path, ir_path, rgb_test_ds.take(10), ir_test_ds.take(10), y[:80])

In [108]:
RGB_MODEL = tf.keras.models.load_model(rgb_path, custom_objects={'r2': r2})
IR_MODEL = tf.keras.models.load_model(ir_path, custom_objects={'r2': r2})

In [109]:
models = [('RGB', RGB_MODEL), ('IR', IR_MODEL)]

In [113]:
rgb_pred = RGB_MODEL.predict(rgb_test_ds.take(10))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
     10/Unknown - 30s 3s/step

2023-03-07 08:35:35.127887: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


10/10 [==============================] - 33s 3s/step


In [114]:
ir_pred = IR_MODEL.predict(ir_test_ds.take(10))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
     10/Unknown - 34s 3s/step

2023-03-07 08:36:12.347171: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


10/10 [==============================] - 39s 4s/step


In [115]:
combined_pred = np.concatenate((rgb_pred, ir_pred), axis=1)
combined_pred.shape

(80, 2)

In [119]:
combined = StackingRegressor(
    models,
    final_estimator=LinearRegression(), #default: RidgeCV,
    cv='prefit',
    n_jobs=-1,
    passthrough=False, #train only on predictions of the models
    verbose=1
)

NameError: name 'test_ds' is not defined

In [120]:
combined.predict(test_ds.take(10))

AttributeError: 'StackingRegressor' object has no attribute 'final_estimator_'

---  
# garbage

In [65]:
rgb_pred

array([[ 0.5475285 ],
       [ 0.18465823],
       [ 0.02080154],
       [-0.10130769],
       [ 0.4011447 ],
       [ 0.11921212],
       [-0.21970168],
       [ 0.5858166 ],
       [ 0.07902738],
       [-0.01635253],
       [ 0.98250365],
       [ 0.90738785],
       [ 0.71365225],
       [ 0.63981485],
       [ 0.42567366],
       [ 0.08188117],
       [-0.14086536],
       [ 0.3389808 ],
       [-0.06948546],
       [ 0.18593866],
       [-0.13502754],
       [ 0.7876036 ],
       [-0.07954195],
       [-0.11578985],
       [ 0.03519647],
       [-0.36530963],
       [-0.2689612 ],
       [-0.26425913],
       [ 0.4567327 ],
       [-0.12853873],
       [ 0.10987496],
       [-0.09405771],
       [ 0.31313795],
       [ 1.3600127 ],
       [-0.497757  ],
       [-0.2301259 ],
       [-0.09953845],
       [-0.39143142],
       [-0.26733506],
       [-0.20070916]], dtype=float32)

In [66]:
ir_pred

array([[ 0.9932999 ],
       [ 0.6453264 ],
       [ 0.31850442],
       [ 1.0797384 ],
       [ 1.1397508 ],
       [ 1.3244345 ],
       [ 0.30700073],
       [-0.09400642],
       [ 0.66349417],
       [ 0.59692097],
       [ 0.0663415 ],
       [ 0.7776587 ],
       [ 1.4182178 ],
       [ 1.1194715 ],
       [ 1.110695  ],
       [ 1.1379107 ],
       [-0.2829012 ],
       [-0.197734  ],
       [ 0.64405   ],
       [-0.21830606],
       [ 0.5146208 ],
       [ 0.20952913],
       [-0.2704628 ],
       [-0.16715337],
       [ 0.22465071],
       [-0.2625861 ],
       [ 0.18379661],
       [-0.12965438],
       [-0.08896695],
       [-0.439563  ],
       [ 0.04323836],
       [-0.22492337],
       [ 0.20035908],
       [ 0.42720315],
       [-0.32625848],
       [-0.16508822],
       [-0.2192713 ],
       [-0.11563718],
       [-0.11729966],
       [-0.33224595]], dtype=float32)

In [32]:
# rgb_pred_flat = rgb_pred.flatten()
# ir_pred_flat = ir_pred.flatten()

In [71]:
mean_pred = np.mean(combined_pred, axis=1)

In [80]:
error = y - mean_pred

In [81]:
mse = np.mean((error ** 2))

In [83]:
mse

0.8164293025061177